# Logging Keras Models with mlflow

* Author: Johannes Maucher
* Last Update: 04.04.2022

**Goal:** In this notebook the logging of Keras Machine Learning Models with [mlflow](https://mlflow.org/docs/latest/index.html) shall be demonstrated. For this we apply a text classification task on the Reuter's newswire.

In [1]:
import numpy as np
import tensorflow.keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import utils
import warnings
warnings.filterwarnings("ignore")

In [2]:
# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow
import mlflow.keras
mlflow.keras.autolog()

## Data Access and split in training- and test-partition

In [3]:
max_words = 1000 # only this number of most frequent words in the corpus are regarded (vocabulary-size)
batch_size = 32
epochs = 5

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

Loading data...


In [5]:
# restore np.load for future normal usage
#np.load = np_load_old

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

8982 train sequences
2246 test sequences
46 classes


Texts are already provided as lists of indicees - each index uniquely identifies a word. The first 10 words of the first text in the trainingset are: 

In [6]:
x_train[0][:10]

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207]

## Create Bag-of-Words matrix

In [7]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)


## Set mlflow experiment

In [8]:
mlflow.set_experiment("Reuters Newswire Topic Classification")

MlflowException: Invalid experiment ID: '.ipynb_checkpoints'

## Define Architecture and Training of Neural Network

In [ ]:
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from matplotlib import pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
max_val_acc=np.max(val_acc)

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(10,8))
plt.grid(True)
plt.plot(epochs, acc, 'bo-', label='Training accuracy')
plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig("reutersMLPclassification.png")
plt.show()

In [ ]:
mlflow.log_artifact("reutersMLPclassification.png")

In [ ]:
mlflow.active_run().info

In [ ]:
mlflow.end_run()